# Optimal tau as a function of the image and noise level

In [ ]:
import time
from collections import namedtuple

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

import scipy.ndimage

try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker

#from ipywidgets import *
try:
    import cv2
except:
    !pip install cv2
    import cv2
    
#import kernels
try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

try:
    import information_theory as IT
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    import information_theory as IT

import utils

In [ ]:
# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

In [ ]:
try:
    import denoising.image.gaussian as denoising
except:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
    import denoising.image.gaussian as denoising

In [ ]:
denoiser = denoising.Monochrome_Denoising(logger)

### lake

In [ ]:
Args = namedtuple("args", ["input"])
#args = Args("http://www.hpca.ual.es/~vruiz/images/barb.png")
#args = Args("http://www.hpca.ual.es/~vruiz/images/zelda.png")
#args = Args("http://www.hpca.ual.es/~vruiz/images/boats.png")
args = Args("http://www.hpca.ual.es/~vruiz/images/lake.png")

In [ ]:
X = skimage_io.imread(args.input)  # Ground Truth

### barb_GF_optimal_tau

In [ ]:
taus = [(0.2 + i/40) for i in range(40)] # Number of points per line
std_devs = [i for i in range(0, 100, 20)]
PCC_curves = []
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    PCC_curve = []
    Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
    for tau in taus: # Filter length
        sigma = np.array([tau, tau])
        kernel = [None]*2
        kernel[0] = utils.get_gaussian_kernel(sigma[0])
        kernel[1] = utils.get_gaussian_kernel(sigma[1])
        #print("Kernel:", kernel)
        denoised = denoiser.filter(Y, kernel)
        PCC = np.corrcoef(denoised.flatten(), X.flatten())[0, 1]
        print("std_dev:", std_dev, "tau:", tau, "PCC:", PCC)
        PCC_curve.append(PCC)

    PCC_curves.append(PCC_curve)
    print()

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
#for i in range(len(curves)):
i = 0
for std_dev in std_devs:
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}, \lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
    #if ((10+i*5) == 40) and ((10-i)/40 == 0.15):
    #plt.plot(sigmas_kernel, curves[i], label=rf"$\sigma={10+i*5}, \gamma={(10-i)/40}" + r", \mathrm{argmax}_\tau=" + rf"{sigmas_kernel[np.argmax(curves[i])]:.2f}$", marker='o')
    #else:
    plt.plot(taus,
        PCC_curves[i],
        label=rf"$\sigma={std_dev}, \gamma={gamma}"
        + r", \tau^*="
        + rf"{taus[np.argmax(PCC_curves[i])]:.2f}"
        + "$")
    i += 1
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}, "
string += r"\mathrm{GF}_\tau"
string += r"(\mathbf{X}"
string += r"+ \mathbf{N}_{\mathcal{N}"
string += r"(\sigma)}"
string += r"+ \mathbf{N}_"
string += r"{\mathcal{P}(\gamma\mathbf{X})}/\gamma))"
string += r"$"
plt.ylabel(string)
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
### barb_GF_optimal_tau
#plt.title(r"$\mathbf{X}=\text{Lake}$")
plt.savefig("lake_GF_optimal_tau.pdf", bbox_inches='tight')